# Analisis exploratorio inicial de datos de CAF

### 2. Proceso ETL
- Carga de datos de parquet a pandas
- Limpieza inicial de los datos
- Tratamiento de datos categoricos
- Tratamiento del campo fecha
- Convertir el campo indice en fecha
- Comprobacion de la frequencia de muestreo
- Ajuste del dataset a la frequencia de muestreo

### V2
- Deteccion del tren parado
- Pendientes
- Datos climatologicos



In [ ]:
!pip install git+https://github.com/maguelo/cafcoding.git#egg=cafcoding

     |████████████████████████████████| 112kB 6.8MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 7.4MB 6.7MB/s 
     |████████████████████████████████| 1.3MB 41.9MB/s 
     |████████████████████████████████| 153kB 45.4MB/s 
     |████████████████████████████████| 143kB 50.7MB/s 
     |████████████████████████████████| 296kB 41.4MB/s 
  Created wheel for aiobotocore: filename=aiobotocore-1.3.0-cp37-none-any.whl size=45781 sha256=28eb09b2b53abb9a7f3b6ad20b30d4e7d8f5130fc070216b50164485ff62a635
  Stored in directory: /root/.cache/pip/wheels/1d/5b/92/72a47210109971364c619c41e29289d4e7d58269f0cc653bf1
Successfully built aiobotocore
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3

In [ ]:
import pyarrow.parquet as pq
import pandas as pd
import sys
import colorlog


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from cafcoding.tools import aws
from cafcoding.tools import meteo
from cafcoding.tools import etl
import cafcoding.tools.log as log
from cafcoding.stages import etl_step
import logging
logger = log.get_logger()
#logger.setLevel(logging.INFO)


# Cargamos la sesion de Amazon

In [ ]:

amazon = aws.AWS("/content/drive/MyDrive/CAFcoding/config/awscli.ini")

# Activamos las credenciales
amazon.set_credentials_in_env()

/content/drive/MyDrive/CAFcoding/config/awscli.ini


In [ ]:

for filename in amazon.ls('cafcodingdatos/raw'):
    print (filename)


cafcodingdatos/raw/Unsaved
cafcodingdatos/raw/_SUCCESS
cafcodingdatos/raw/date=2020-10-01
cafcodingdatos/raw/date=2020-10-02
cafcodingdatos/raw/date=2020-10-03
cafcodingdatos/raw/date=2020-10-04
cafcodingdatos/raw/date=2020-10-05
cafcodingdatos/raw/date=2020-10-06
cafcodingdatos/raw/date=2020-10-07
cafcodingdatos/raw/date=2020-10-08
cafcodingdatos/raw/date=2020-10-09
cafcodingdatos/raw/date=2020-10-10
cafcodingdatos/raw/date=2020-10-11
cafcodingdatos/raw/date=2020-10-12
cafcodingdatos/raw/date=2020-10-13
cafcodingdatos/raw/date=2020-10-14
cafcodingdatos/raw/date=2020-10-15
cafcodingdatos/raw/date=2020-10-16
cafcodingdatos/raw/date=2020-10-17
cafcodingdatos/raw/date=2020-10-18
cafcodingdatos/raw/date=2020-10-19
cafcodingdatos/raw/date=2020-10-20
cafcodingdatos/raw/date=2020-10-21
cafcodingdatos/raw/date=2020-10-22
cafcodingdatos/raw/date=2020-10-23
cafcodingdatos/raw/date=2020-10-24
cafcodingdatos/raw/date=2020-10-25
cafcodingdatos/raw/date=2020-10-26
cafcodingdatos/raw/date=2020-10-27


# Cargamos el dataset



In [ ]:
meteo_path = r'drive/MyDrive/CAFcoding/notebook/utils/meteo_data'


parquet_csv_list =[]
GENERATE_TRAIN = True
GENERATE_VAL =False
GENERATE_TEST =True
NUMBER_DAYS = 1

PATH_TEMPLATE='s3://cafcodingdatos/targets_mirror/{step}/2020-{month:02d}-{day:02d}-etl-{version}.csv'

if GENERATE_TRAIN:
    STEP = 'train'
    MONTH = 10
    NUMBER_DAYS = 31
    START_DATE = 1 
    parquet_paths_s3 = ['cafcodingdatos/raw/date=2020-10-{0:02d}'.format(day) for day in range(START_DATE,NUMBER_DAYS+1)]
    csv_path  = [PATH_TEMPLATE.format(step=STEP,month=MONTH, day=day,version=etl_step.ETL_VERSION) for day in range(START_DATE,NUMBER_DAYS+1)]


    parquet_csv_list += list(zip(parquet_paths_s3,csv_path))

if GENERATE_VAL:
    STEP = 'val'
    MONTH = 11
    #NUMBER_DAYS = 10
    START_DATE = 1
    parquet_paths_s3 = ['cafcodingdatos/raw/date=2020-11-{0:02d}'.format(day) for day in range(START_DATE,NUMBER_DAYS+1)]
    csv_path  = [PATH_TEMPLATE.format(step=STEP,month=MONTH, day=day,version=etl_step.ETL_VERSION) for day in range(START_DATE,NUMBER_DAYS+1)]

    parquet_csv_list += list(zip(parquet_paths_s3,csv_path))

if GENERATE_TEST:
    STEP = 'test'
    MONTH = 11
    NUMBER_DAYS = 30
    START_DATE = 11
    parquet_paths_s3 = ['cafcodingdatos/raw/date=2020-11-{0:02d}'.format(day) for day in range(START_DATE,NUMBER_DAYS+1)]
    csv_path  = [PATH_TEMPLATE.format(step=STEP,month=MONTH, day=day,version=etl_step.ETL_VERSION) for day in range(START_DATE,NUMBER_DAYS+1)]

    parquet_csv_list += list(zip(parquet_paths_s3,csv_path))





# Funciones personalizadas

De esta forma se pueden probar funciones sin recargar el notebook


In [ ]:
# def delete_stopped_train(df):
#     print ("test")
#     return df

# etl_step.delete_stopped_train = delete_stopped_train

### Cargamos datos desde S3


In [ ]:
df_meteo = meteo.load_meteo_dataframe(meteo_path)

df_list = []
for parquet_file, csv_file in parquet_csv_list:
    df_day_list=[]
    for filename in amazon.ls(parquet_file):
        if filename.endswith('.parquet'):
            try:
                print (filename)
                dataset = pq.ParquetDataset(filename, filesystem=amazon.fs)
                table = dataset.read()
                df = table.to_pandas() 
                df_day_list.append(df)
            except Exception as exc:
                logger.error("Se ha producido un error cargando el fichero de Parquet: {}",parquet_file)
                logger.error("{}",str(exc))
            
    
    df_day = pd.concat(df_day_list)
    try:
        df_day = etl_step.process_etl(df_day, df_meteo.copy(), del_stopped_train=True)
        df_day.to_csv(csv_file, index=True, index_label="ts_date_index")
    except Exception as exc:
        logger.error("Se ha producido un error al procesar el dataset: {} ",parquet_file)
        logger.error("{}",str(exc))
    df_list.append(df_day)




df = pd.concat(df_list)

cafcodingdatos/raw/date=2020-10-04/part-00017-tid-4502735041458916000-1def1e63-76d9-4841-bd06-f798a2422800-4199-4.c000.snappy.parquet
cafcodingdatos/raw/date=2020-10-04/part-00055-tid-4502735041458916000-1def1e63-76d9-4841-bd06-f798a2422800-4147-3.c000.snappy.parquet
cafcodingdatos/raw/date=2020-10-04/part-00091-tid-4502735041458916000-1def1e63-76d9-4841-bd06-f798a2422800-4237-3.c000.snappy.parquet
cafcodingdatos/raw/date=2020-10-04/part-00108-tid-4502735041458916000-1def1e63-76d9-4841-bd06-f798a2422800-4189-2.c000.snappy.parquet
cafcodingdatos/raw/date=2020-10-04/part-00122-tid-4502735041458916000-1def1e63-76d9-4841-bd06-f798a2422800-4136-2.c000.snappy.parquet
cafcodingdatos/raw/date=2020-10-04/part-00142-tid-4502735041458916000-1def1e63-76d9-4841-bd06-f798a2422800-4234-3.c000.snappy.parquet
cafcodingdatos/raw/date=2020-10-04/part-00162-tid-4502735041458916000-1def1e63-76d9-4841-bd06-f798a2422800-4152-3.c000.snappy.parquet
 [Sun, 18 Apr 2021 23:59:20] DEBUG [log.py.log_decorator_wrapp

DEBUG:ETL:<function generate_auxiliar_columns at 0x7f0cb0022200> END


 [Mon, 19 Apr 2021 00:04:04] DEBUG [log.py.log_decorator_wrapper:38] <function create_categorical at 0x7f0cb0022560> START


DEBUG:ETL:<function create_categorical at 0x7f0cb0022560> START


 [Mon, 19 Apr 2021 00:04:05] DEBUG [log.py.log_decorator_wrapper:41] <function create_categorical at 0x7f0cb0022560> END


...

DEBUG:ETL:<function generate_new_columns_from_gps_data at 0x7f0cb001ee60> START


 [Mon, 19 Apr 2021 02:41:46] DEBUG [etl.py.fill_dataframe_by_ut:105] In: 2020-10-25 


DEBUG:ETL:In: 2020-10-25 


 [Mon, 19 Apr 2021 02:41:47] DEBUG [etl.py.fill_dataframe_by_ut:116] Out: 2020-10-25 


DEBUG:ETL:Out: 2020-10-25 


In [ ]:
df.head()